In [ ]:
import pandas as pd
import matplotlib.pyplot as pt
import numpy as np
import os
import seaborn as sns

data = pd.read_excel('ETR-259 Findings.xlsm', sheet_name=None, index_col=False)
df = pd.concat(data.values(), sort=True, ignore_index=True)

len(df)

df = df.iloc[1:]

df.head(15)

#df = df.reset_index(drop=True)

In [ ]:
df.columns = df.iloc[9,:]
df.head()

In [ ]:
average = df[df.values == "Average"]
std = df[df.values == "Std Dev"]
dataRef = df[df.values =="Data ref"]

average_list = average.index
std_list = std.index
dataRef_list = dataRef.index

df = df.drop(index = average_list)
df = df.drop(index = std_list)
df = df.drop(index = dataRef_list)

df = df.reset_index(drop=True)
df.head(15)

In [ ]:
df.columns

In [ ]:
df["DataRef"] = np.nan
for i in range(1,19):
    df["DataRef"] = df["DataRef"].fillna(df.iloc[:,i])
    
df.head(20)

In [ ]:
df = df.iloc[:,18:]
df.head()

In [ ]:
df.dropna(axis = 0, inplace=True)

In [ ]:
df.head(15)

In [ ]:
df = df.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna();

In [ ]:
df.drop(['Pcyl 1 @ BDC', 'SOI (cyl 1), CA aTDC', 'Pi, (kW)', 'Pf, (kW)',
       'Pp, (kW)', 'k1 (0% HEF)',
       'k1 (50% HEF)', 'k1 (100% HEF)', 'Mechanical efficiency', 'T, HEF Out (degC)','T, Oil (degC)','Pmax (cyl 1)','DataRef'], axis = 1, inplace=True);

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plot = plt.figure(figsize = (18,18))
sns.heatmap(df.corr(), annot=True, linewidths=0.5, cmap="BrBG", square=True)

In [ ]:
df.plot.scatter(x = 'N2 MassFlow Coriolis (kg/min)', y = 'HS_TransPower (kW)', figsize = (12,6))

In [ ]:
df[df['P, N2 (bar)'] < 35].plot.scatter(x = 'N2 MassFlow Coriolis (kg/min)', y = 'BSFC_Trans_Coriolis (Wh/kg)', figsize=(12,6))

## Graph above shows that the Power output increases with increasing N2 mass flow and that it is linear. So linear model could be used.
### Interestingly the flow rate is independent of RPM and N2 pressure

In [ ]:
df.head()

## The columns I want to keep
* Power
* BSFC
* N2 pressure
* N2 flow
* T, HEF
* Engine Speed

In [ ]:
df.plot.scatter(x = 'N2 MassFlow Coriolis (kg/min)', y = 'HS_TransPower (kW)', figsize = (12,6))

In [ ]:
df.columns

In [ ]:
df_filtered = df.filter(['P, N2 (bar)', 'BSFC_Trans_Coriolis (Wh/kg)','N2 MassFlow Coriolis (kg/min)','T, HEF Inlet(degC)','Engine speed (rpm)','HS_TransPower (kW)'], axis =1)
df_filtered.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plot = plt.figure(figsize = (18,18))
sns.heatmap(df_filtered.corr(), annot=True, linewidths=0.5, cmap="BrBG", square=True)

In [ ]:
df_filtered.isnull().sum()

In [ ]:
indices = range(len(df_filtered))
train = [indices[i] for i in indices if i%3 !=0]
test = [indices[i] for i in indices if i%3 ==0]
test_df = df_filtered.iloc[test]
train_df =df_filtered.iloc[train]

In [ ]:
train_df.columns

In [ ]:
test_df.loc[len(test_df)] = [0 ,0,0,0,0,8] 
test_df

In [ ]:
from sklearn.linear_model import LinearRegression

x_train = train_df.drop(["N2 MassFlow Coriolis (kg/min)", "BSFC_Trans_Coriolis (Wh/kg)"], axis =1)
y_train = train_df["N2 MassFlow Coriolis (kg/min)"]
x_test = test_df.drop(["N2 MassFlow Coriolis (kg/min)", "BSFC_Trans_Coriolis (Wh/kg)"], axis=1)
x_train.shape, y_train.shape, x_test.shape

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
Y_pred = lr.predict(x_test)

difference = 100*(((Y_pred)-test_df["N2 MassFlow Coriolis (kg/min)"])/(Y_pred))

predicted_power = pd.DataFrame({
        "HEF Temp. [°C]": test_df["T, HEF Inlet(degC)"],
        "Engine Speed [RPM]": test_df["Engine speed (rpm)"],
        "N2 Pressure [barg]": test_df["P, N2 (bar)"],
        #"BSFC [Wh/kg]": test_df["BSFC_Trans_Coriolis (Wh/kg)"],
        "Break Power [W]":test_df["HS_TransPower (kW)"],
        "N2 MassFlow Coriolis (kg/min)": test_df["N2 MassFlow Coriolis (kg/min)"],
        "N2 MassFlow Coriolis (kg/min)_predicted":Y_pred,
        "Difference": difference
    })

#predicted_power.to_csv('LinearRegression.csv', index=False)
#print("The maximum percentage difference is:", difference.max())
#print ("The min percentage difference is:", difference.min())
#predicted_power.to_csv('N2FlowPredicted.csv')


In [ ]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [ ]:
print (style.BOLD + "The N2 flow rate to achieve 8kw Power output is predicted to be"+ style.END, predicted_power["N2 MassFlow Coriolis (kg/min)_predicted"].max() )

In [ ]:
print ("The N2 flow rate to achieve 8kw Power output is predicted to be", predicted_power["N2 MassFlow Coriolis (kg/min)_predicted"].max())